In [21]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [22]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [23]:
tf.keras.backend.clear_session()

In [24]:
subject = 'Distrito Federal - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [25]:
data = pd.read_csv('2003_mo_model_input_DF.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t),Distrito Federal - Consumo de Cimento (t)
0,2003-1,0.254227,8.293882,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.826971,1.184017e+08,3.570616e+06,43.639430,1.092465e+08,156.625720,50.047
1,2003-2,0.252655,8.287887,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.827063,1.185511e+08,3.573184e+06,43.651300,1.093025e+08,157.678528,48.110
2,2003-3,0.250972,8.281892,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.827155,1.187006e+08,3.575752e+06,43.663170,1.093585e+08,159.190268,49.006
3,2003-4,0.249179,8.275896,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.827247,1.188500e+08,3.578320e+06,43.675041,1.094145e+08,160.688376,48.445
4,2003-5,0.247070,8.269901,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.827340,1.189994e+08,3.580889e+06,43.686911,1.094705e+08,162.847410,51.436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.529279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250.077918,77.448
236,2022-9,0.527896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,248.545664,64.321
237,2022-10,0.526069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,248.265413,72.839
238,2022-11,0.523943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247.486640,59.598


In [26]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
0,-0.765895,-0.830412,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.593530,-1.693822,0.396337,-2.171734,-2.205731,-1.931775
1,-0.800114,-0.833674,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.541888,-1.674984,0.431282,-2.081307,-2.163694,-1.908347
2,-0.836745,-0.836936,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.490246,-1.656145,0.466227,-1.990880,-2.121657,-1.874707
3,-0.875764,-0.840198,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.438604,-1.637307,0.501171,-1.900453,-2.079620,-1.841370
4,-0.921678,-0.843461,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.386962,-1.618468,0.536116,-1.810026,-2.037583,-1.793327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.298634,1.198041,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.236110,1.116166,-1.071950,-1.286229,0.817896,-0.512069
188,1.354774,1.198255,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.183845,1.104850,-1.047047,-1.328476,0.799100,-0.500824
189,1.401307,1.198470,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.131580,1.093534,-1.022144,-1.370723,0.780305,-0.491677
190,1.460103,1.198684,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.079315,1.082218,-0.997242,-1.412970,0.761510,-0.481495


In [27]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      46.442
1      43.234
2      54.587
3      59.228
4      63.997
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Distrito Federal - Consumo de Cimento (t), Length: 240, dtype: float64

In [28]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
0,-0.765895,-0.830412,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.593530,-1.693822,0.396337,-2.171734,-2.205731,-1.931775
1,-0.800114,-0.833674,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.541888,-1.674984,0.431282,-2.081307,-2.163694,-1.908347
2,-0.836745,-0.836936,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.490246,-1.656145,0.466227,-1.990880,-2.121657,-1.874707
3,-0.875764,-0.840198,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.438604,-1.637307,0.501171,-1.900453,-2.079620,-1.841370
4,-0.921678,-0.843461,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.386962,-1.618468,0.536116,-1.810026,-2.037583,-1.793327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.937260,1.309227,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.616226,1.235244,-1.506889,-0.747477,1.174551,0.131039
158,0.925760,1.302282,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.563561,1.236423,-1.511843,-0.763990,1.168128,0.077457
159,0.912813,1.295337,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.510895,1.237602,-1.516798,-0.780503,1.161706,0.032090
160,0.906620,1.288392,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.458230,1.238780,-1.521752,-0.797016,1.155283,-0.032489


In [29]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      46.442
1      43.234
2      54.587
3      59.228
4      63.997
        ...  
157    35.599
158    49.981
159    39.881
160    48.598
161    49.942
Name: Distrito Federal - Consumo de Cimento (t), Length: 162, dtype: float64

In [30]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [31]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [32]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
126,0.746963,0.952687,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.686090,0.806860,-0.638912,-0.325093,0.894339,1.712291
127,0.759387,0.973455,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.670518,0.828599,-0.679356,-0.322764,0.917121,1.675940
128,0.775249,0.994223,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.654947,0.850339,-0.719800,-0.320434,0.939903,1.626852
129,0.795874,1.014990,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.639375,0.872078,-0.760244,-0.318105,0.962685,1.569267
130,0.815598,1.035758,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.623804,0.893818,-0.800689,-0.315775,0.985467,1.514446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.298634,1.198041,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.236110,1.116166,-1.071950,-1.286229,0.817896,-0.512069
188,1.354774,1.198255,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.183845,1.104850,-1.047047,-1.328476,0.799100,-0.500824
189,1.401307,1.198470,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.131580,1.093534,-1.022144,-1.370723,0.780305,-0.491677
190,1.460103,1.198684,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.079315,1.082218,-0.997242,-1.412970,0.761510,-0.481495


In [33]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [34]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [35]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [36]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [37]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2877767848, 2864940106, 3777401102, 3217099576, 2386823331, 1071767361, 4012120043, 93953697, 51726945, 1762626918, 1453246222, 1941220400, 2563620314, 1757853228, 3341545102, 1786197225, 390786602, 1094939534, 984123735, 1131984387, 1746432735, 3097406738, 58021491, 3107125007, 1851471069, 2910366370, 1885724688, 1402595869, 976004864, 3704903948, 3835324661, 152056309, 1931013393, 1381282465, 4247039776, 1110084574, 665458243, 553973662, 78661257, 2294197307, 4273937354, 3629564418, 1133330092, 3013712045, 3676663049, 2128053363, 803521794, 310244227, 1244080819, 693577995]


Step: 0 ___________________________________________
val_loss: 26.15943145751953
winner_seed: 2877767848


Step: 1 ___________________________________________
val_loss: 24.30448341369629
winner_seed: 2864940106


Step: 2 ___________________________________________
val_loss: 31.76424217224121


Step: 3 ___________________________________________
val_loss: 31.052764892578125


Step: 4 _____________________________

In [38]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 131ms/step - loss: 5594.4663 - val_loss: 529.5046
Epoch 2/10000
4/4 [==============================] - 0s 14ms/step - loss: 429.9200 - val_loss: 579.0449
Epoch 3/10000
4/4 [==============================] - 0s 13ms/step - loss: 431.3725 - val_loss: 838.0822
Epoch 4/10000
4/4 [==============================] - 0s 12ms/step - loss: 799.0927 - val_loss: 593.2561
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 532.8136 - val_loss: 383.2176
Epoch 6/10000
4/4 [==============================] - 0s 15ms/step - loss: 270.9914 - val_loss: 340.7278
Epoch 7/10000
4/4 [==============================] - 0s 12ms/step - loss: 467.0750 - val_loss: 673.3160
Epoch 8/10000
4/4 [==============================] - 0s 13ms/step - loss: 494.3963 - val_loss: 514.7316
Epoch 9/10000
4/4 [==============================] - 0s 13ms/step - loss: 309.4019 - val_loss: 313.2077
Epoch 10/10000
4/4 [==============================] - 0s 12ms/

4/4 [==============================] - 0s 13ms/step - loss: 85.7038 - val_loss: 120.1650
Epoch 80/10000
4/4 [==============================] - 0s 12ms/step - loss: 71.7290 - val_loss: 130.7098
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 77.5117 - val_loss: 118.9588
Epoch 82/10000
4/4 [==============================] - 0s 12ms/step - loss: 77.7927 - val_loss: 136.2105
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 77.9771 - val_loss: 120.2746
Epoch 84/10000
4/4 [==============================] - 0s 13ms/step - loss: 73.2823 - val_loss: 113.5444
Epoch 85/10000
4/4 [==============================] - 0s 13ms/step - loss: 79.7895 - val_loss: 1133.8430
Epoch 86/10000
4/4 [==============================] - 0s 13ms/step - loss: 1688.9761 - val_loss: 378.7497
Epoch 87/10000
4/4 [==============================] - 0s 11ms/step - loss: 285.3961 - val_loss: 221.1841
Epoch 88/10000
4/4 [==============================] - 0s 11ms/step - loss: 

4/4 [==============================] - 0s 9ms/step - loss: 77.1128 - val_loss: 114.8829
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 96.2001 - val_loss: 122.0997
Epoch 159/10000
4/4 [==============================] - 0s 9ms/step - loss: 73.5802 - val_loss: 105.3857
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 74.5387 - val_loss: 109.7017
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 71.0245 - val_loss: 129.5881
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 82.6159 - val_loss: 111.8904
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 69.7366 - val_loss: 103.0105
Epoch 164/10000
4/4 [==============================] - 0s 8ms/step - loss: 68.7881 - val_loss: 109.7001
Epoch 165/10000
4/4 [==============================] - 0s 8ms/step - loss: 93.5189 - val_loss: 120.2712
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - loss: 76.

4/4 [==============================] - 0s 9ms/step - loss: 51.1241 - val_loss: 87.0692
Epoch 237/10000
4/4 [==============================] - 0s 8ms/step - loss: 47.8663 - val_loss: 78.8370
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 50.2821 - val_loss: 82.0063
Epoch 239/10000
4/4 [==============================] - 0s 9ms/step - loss: 49.2377 - val_loss: 76.6521
Epoch 240/10000
4/4 [==============================] - 0s 12ms/step - loss: 44.3419 - val_loss: 85.1485
Epoch 241/10000
4/4 [==============================] - 0s 12ms/step - loss: 60.5255 - val_loss: 113.3163
Epoch 242/10000
4/4 [==============================] - 0s 12ms/step - loss: 59.3394 - val_loss: 121.3744
Epoch 243/10000
4/4 [==============================] - 0s 12ms/step - loss: 64.8856 - val_loss: 113.1090
Epoch 244/10000
4/4 [==============================] - 0s 11ms/step - loss: 64.2880 - val_loss: 93.9937
Epoch 245/10000
4/4 [==============================] - 0s 10ms/step - loss: 58.17

Epoch 315/10000
4/4 [==============================] - 0s 12ms/step - loss: 55.0680 - val_loss: 89.5655
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 51.3405 - val_loss: 92.6679
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 55.7402 - val_loss: 81.3663
Epoch 318/10000
4/4 [==============================] - 0s 13ms/step - loss: 49.5556 - val_loss: 85.3127
Epoch 319/10000
4/4 [==============================] - 0s 12ms/step - loss: 49.7016 - val_loss: 100.5249
Epoch 320/10000
4/4 [==============================] - 0s 12ms/step - loss: 63.2854 - val_loss: 88.8795
Epoch 321/10000
4/4 [==============================] - 0s 12ms/step - loss: 61.8966 - val_loss: 110.7826
Epoch 322/10000
4/4 [==============================] - 0s 12ms/step - loss: 76.4232 - val_loss: 86.9525
Epoch 323/10000
4/4 [==============================] - 0s 13ms/step - loss: 68.3680 - val_loss: 76.7268
Epoch 324/10000
4/4 [==============================] - 0s 13ms

...

4/4 [==============================] - 0s 14ms/step - loss: 17.9261 - val_loss: 29.6676
Epoch 1103/10000
4/4 [==============================] - 0s 12ms/step - loss: 13.8034 - val_loss: 26.7048
Epoch 1104/10000
4/4 [==============================] - 0s 11ms/step - loss: 10.7695 - val_loss: 27.0606
Epoch 1105/10000
4/4 [==============================] - 0s 13ms/step - loss: 11.3251 - val_loss: 24.5350
Epoch 1106/10000
4/4 [==============================] - 0s 13ms/step - loss: 13.8560 - val_loss: 24.9822
Epoch 1107/10000
4/4 [==============================] - 0s 14ms/step - loss: 14.2410 - val_loss: 25.2067
Epoch 1108/10000
4/4 [==============================] - 0s 12ms/step - loss: 14.2899 - val_loss: 24.8382
Epoch 1109/10000
4/4 [==============================] - 0s 12ms/step - loss: 10.6730 - val_loss: 26.5482
Epoch 1110/10000
4/4 [==============================] - 0s 12ms/step - loss: 9.1474 - val_loss: 28.1264
Epoch 1111/10000
4/4 [==============================] - 0s 14ms/step - lo

4/4 [==============================] - 0s 12ms/step - loss: 12.8353 - val_loss: 25.7983
Epoch 1181/10000
4/4 [==============================] - 0s 12ms/step - loss: 13.4299 - val_loss: 25.0362
Epoch 1182/10000
4/4 [==============================] - 0s 11ms/step - loss: 10.0680 - val_loss: 25.8528
Epoch 1183/10000
4/4 [==============================] - 0s 11ms/step - loss: 13.2743 - val_loss: 25.5717
Epoch 1184/10000
4/4 [==============================] - 0s 13ms/step - loss: 14.6934 - val_loss: 30.1274
Epoch 1185/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.2413 - val_loss: 26.5542
Epoch 1186/10000
4/4 [==============================] - 0s 12ms/step - loss: 10.1930 - val_loss: 26.4085
Epoch 1187/10000
4/4 [==============================] - 0s 12ms/step - loss: 8.2884 - val_loss: 32.5005
Epoch 1188/10000
4/4 [==============================] - 0s 12ms/step - loss: 11.6494 - val_loss: 26.9704
Epoch 1189/10000
4/4 [==============================] - 0s 12ms/step - lo

4/4 [==============================] - 0s 12ms/step - loss: 11.0816 - val_loss: 27.3531
Epoch 1259/10000
4/4 [==============================] - 0s 14ms/step - loss: 12.7002 - val_loss: 29.6748
Epoch 1260/10000
4/4 [==============================] - 0s 15ms/step - loss: 12.7321 - val_loss: 25.0973
Epoch 1261/10000
4/4 [==============================] - 0s 13ms/step - loss: 12.0224 - val_loss: 29.1333
Epoch 1262/10000
4/4 [==============================] - 0s 15ms/step - loss: 11.8880 - val_loss: 29.6144
Epoch 1263/10000
4/4 [==============================] - 0s 12ms/step - loss: 9.8221 - val_loss: 27.7932
Epoch 1264/10000
4/4 [==============================] - 0s 12ms/step - loss: 9.2585 - val_loss: 25.8550
Epoch 1265/10000
4/4 [==============================] - 0s 12ms/step - loss: 9.8171 - val_loss: 28.4836
Epoch 1266/10000
4/4 [==============================] - 0s 11ms/step - loss: 10.9352 - val_loss: 25.2756
Epoch 1267/10000
4/4 [==============================] - 0s 12ms/step - loss

4/4 [==============================] - 0s 13ms/step - loss: 10.9920 - val_loss: 36.0549
Epoch 1337/10000
4/4 [==============================] - 0s 12ms/step - loss: 12.0590 - val_loss: 30.3320
Epoch 1338/10000
4/4 [==============================] - 0s 13ms/step - loss: 10.4393 - val_loss: 27.2796
Epoch 1339/10000
4/4 [==============================] - 0s 12ms/step - loss: 10.8484 - val_loss: 28.4190
Epoch 1340/10000
4/4 [==============================] - 0s 12ms/step - loss: 12.5381 - val_loss: 25.5830
Epoch 1341/10000
4/4 [==============================] - 0s 11ms/step - loss: 10.1264 - val_loss: 26.6742
Epoch 1342/10000
4/4 [==============================] - 0s 11ms/step - loss: 12.3539 - val_loss: 31.4022
Epoch 1343/10000
4/4 [==============================] - 0s 12ms/step - loss: 11.4267 - val_loss: 26.3287
Epoch 1344/10000
4/4 [==============================] - 0s 12ms/step - loss: 10.1063 - val_loss: 28.3190
Epoch 1345/10000
4/4 [==============================] - 0s 12ms/step - l

4/4 [==============================] - 0s 12ms/step - loss: 11.7350 - val_loss: 35.7939
Epoch 1415/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.9168 - val_loss: 30.8791
Epoch 1416/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.5674 - val_loss: 30.6901
Epoch 1417/10000
4/4 [==============================] - 0s 14ms/step - loss: 10.1531 - val_loss: 30.1211
Epoch 1418/10000
4/4 [==============================] - 0s 13ms/step - loss: 10.7333 - val_loss: 28.3005
Epoch 1419/10000
4/4 [==============================] - 0s 13ms/step - loss: 10.6700 - val_loss: 28.3399
Epoch 1420/10000
4/4 [==============================] - 0s 13ms/step - loss: 11.7382 - val_loss: 46.5528
Epoch 1421/10000
4/4 [==============================] - 0s 14ms/step - loss: 21.2136 - val_loss: 26.6885
Epoch 1422/10000
4/4 [==============================] - 0s 13ms/step - loss: 10.3964 - val_loss: 28.9822
Epoch 1423/10000
4/4 [==============================] - 0s 13ms/step - l

In [39]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [40]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,46.603226,46.584602,46.441311,46.48267,46.398762,46.206764,46.283852,46.391098,46.381836,46.419598,46.548904,46.728813,46.326504,46.727654,46.828266,46.701286,46.515911,46.314583,46.759171,46.851158,46.798023,46.868126,46.890366,46.921757,46.951145,46.971687,47.044807,47.146008,47.24601,47.283485
Target,53.235,59.29,54.875,57.52,49.573,40.698,47.195,43.527,45.488,44.96,36.721,54.463,51.462,58.854,53.94,51.514,42.635,40.371,48.99,44.726,44.491,43.901,55.019,50.942,63.992,64.127,60.822,65.182,51.282,43.554
Error,6.631775,12.705399,8.433689,11.037331,3.17424,5.508762,0.911148,2.864098,0.893837,1.459599,9.827904,7.734188,5.135498,12.126347,7.111732,4.812714,3.880913,5.943584,2.230831,2.125156,2.307022,2.967125,8.128635,4.020245,17.040855,17.155312,13.777191,18.035992,4.035992,3.729485


In [41]:
display(mae)
display(mape)

6.8582206

0.12801908

In [42]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [43]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined